# 神经风格迁移

* **风格**(style)是指图像中不同空间尺度的纹理\颜色和视觉图案

* **内容**(content)是指图像的高级宏观结构.

实现风格迁移背后的关键概念与所有深度学习算法的核心思想是一样的:
定义一个损失函数制定想要实现的目标,然后将这个损失最小化.

## 内容损失

内容损失的一个很好的候选者就是两个激活之间的L2范数,一个激活是预训练的卷积神经网络更靠顶部的某层在目标图像上计算得到的激活,另一个激活是同一层在生成图像上计算得到的激活.

这可以保证,在更靠近顶部的层看来,生成图像与原始目标图像看起来很相似.


## 风格损失

风格损失的目的是在风格参考图像与生成图像之间,在不同的层激活内保存想死的内部相互关系.这保证了在风格参考图像与生成图像之间,不同空间尺度找到的纹理看起来都很相似.


简而言之,可以使用预训练的卷积神经网络来定义一个具有以下特点的损失

* 在目标内容图像和生成图像之间保持相似的较高层激活,从而能够保留内容.卷积神经网络应该能够"看到"目标图像和生成的图像包含相同的内容

* 在较低层和较高层的激活中保持类似的相互关系(correlation),从而能够保留风格.特征相互关系捕捉到的是纹理(texture),生成图像和风格参考图像在不同的空间尺度上应该具有相同的纹理.

## 用Keras实现神经风格迁移 

使用预训练的VGG19

#### 定义初始变量

In [1]:
import keras
from keras.preprocessing.image import load_img, img_to_array

Using TensorFlow backend.


In [14]:
DATASETS = "../datasets/imgs/"
STYLE_IMG = "../datasets/imgs/style/starry-night.jpg"
CONTENT_IMG = "../datasets/imgs/content/gyeongbokgung.jpg"

In [3]:
# 生成图像的尺寸
width, height = load_img(CONTENT_IMG).size
img_height = 400
img_width = int(width * img_height / height)

In [4]:
print("img_height is %d, img_width is %d" % (img_height, img_width))

img_height is 400, img_width is 602


#### 辅助函数

In [5]:
import numpy as np
from keras.applications import vgg19

def preprocess_image(image_path):
    img = load_img(image_path, target_size = (img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    # 减去ImageNet的平均像素值
    # 使其中心为0,这里相当于vgg19.preprocess_input的逆操作
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 将图像由BGR格式转换为RGB格式
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x
    

#### 加载预训练的VGG19网络,并将其应用于3张图片

In [8]:
from keras import backend as K
content_image = K.constant(preprocess_image(CONTENT_IMG))
style_image = K.constant(preprocess_image(STYLE_IMG))
combination_image = K.placeholder((1, img_height, img_width, 3))

input_tensor = K.concatenate([content_image, style_image, combination_image], axis = 0)

model = vgg19.VGG19(input_tensor = input_tensor, 
                    weights = 'imagenet',
                    include_top = False)

print('Model loaded')

Model loaded


#### 内容损失

In [9]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

#### 风格损失

In [10]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

#### 总变差损失

In [11]:
def total_variation_loss(x):
    a = K.square(
        x[:, :img_height - 1, :img_width - 1, :] -
        x[:, 1:, :img_width - 1, :])
    b = K.square(
        x[:, :img_height - 1, :img_width - 1, :] -
        x[:, img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

需要最小化的损失是这三项损失的加权平均.

为了计算内容损失,只使用一个靠顶部的层,即block5_conv2层;
对于风格损失,需要使用一系列层,既包括顶层也包括底层.
最后还需要添加总变差损失

根据所使用的风格参考图像和内容图像,很可能还需要调节content_weight系数(内容损失对总损失的贡献比例).

更大的content_weight表示目标内容更容易在生成图像中被识别出来

#### 定义需要最小化的最终损失

In [12]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

# 用于内容损失的层
content_layer = 'block5_conv2'

# 用于风格损失的层
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']

# 损失分量的加权平均所使用的权重
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

loss = K.variable(0.)
layer_features = outputs_dict[content_layer]
content_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
# 添加内容损失
loss += content_weight * content_loss(content_image_features, combination_features)

# 添加每个层的风格损失分量
for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    s1 = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(style_layers)) * s1
    
# 添加总变差损失
loss += total_variation_weight * total_variation_loss(combination_image)


#### 设置梯度下降过程

In [13]:
# 获取损失相对于生成图像的梯度
grads = K.gradients(loss, combination_image)[0]

# 获取当前损失值和当前梯度值的函数
fetch_loss_and_grads = K.function([combination_image], [loss, grads])

class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grad_values = None
        
    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value
    
    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values
    
evaluator = Evaluator()

#### 风格迁移循环

In [16]:
from scipy.optimize import fmin_l_bfgs_b
from skimage import io
import time

result_prefix = 'my_result'
iterations = 20

x = preprocess_image(CONTENT_IMG)
x = x.flatten()
for i in range(iterations):
    print('Start of iteration ', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, 
                                     x,
                                     fprime = evaluator.grads,
                                     maxfun = 20)
    print('Current loss value: ', min_val)
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_prefix + '_at_iteration_%d.png' % i
    path = os.path.join(DATASETS, fname)
    io.imsave(path, img)
    end_time = time.time()
    print("Iteration %d completed in %ds" % (i, end_time - start_time))

Start of iteration  0


InternalError: CUB reduce errorout of memory
	 [[Node: Sum_6 = Sum[T=DT_FLOAT, Tidx=DT_INT32, keep_dims=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Pow, Const_18)]]
	 [[Node: add_7/_197 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1010_add_7", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Sum_6', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/d